In [2]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import pickle
import matplotlib.pyplot as plt

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Pose Graph Optimization

### GOAL: Adjust configuration of absolute poses (nodes) to minimize squared error intorduced by constraints (edges)


Node: [x, y, z, r, p, y]

Edge: Odometry Measurements

# State vector $x$

absolute position and orientation of vehicle at each timestep

\begin{equation}
\textbf{x} = 
\begin{bmatrix}
\textbf{x}_1\\
\textbf{x}_2\\
\textbf{x}_3 \\
\vdots
\end{bmatrix} = 
\begin{bmatrix}
x_1 & y_1 & z_1 & r_1 & p_1 & y_1 \\
x_2 & y_2 & z_2 & r_2 & p_2 & y_2 \\
x_3 & y_3 & z_3 & r_3 & p_3 & y_3 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \\ 
\end{bmatrix}
\end{equation}

# Transformations $\mathbf{X}$  
represeneted in $\textit{Homogenous Coordinates}$

\begin{equation}
\mathbf{X_i} = 
\begin{bmatrix}
R_{11} & R_{12} & R_{13} & dx\\
R_{21} & R_{22} & R_{23} & dy \\
R_{31} & R_{32} & R_{33} & dz \\
0 & 0 & 0 & 1
\end{bmatrix} 
\end{equation}


$\big(\mathbf{X}_i^{-1} \mathbf{X}_{i+1} \big)$  describes how node $i$ sees node $(i+1)$ (ex: odometry)

$\big(\mathbf{X}_i^{-1} \mathbf{X}_{j} \big)$  describes how node $i$ sees node $j$ (ex: loop closure)

In [3]:
def v2t(vector):
    """converts a transformation vector to homogenous coordinate system"""
    if len(tf.shape(vector)) == 1: #allow for 1-D or N-D input
        vector = vector[None,:]
    angs = vector[:,3:]
    phi = angs[:,0]
    theta = angs[:,1]
    psi = angs[:,2]
    rot = tf.Variable([[cos(theta)*cos(psi), sin(psi)*cos(phi) + sin(phi)*sin(theta)*cos(psi), sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi)],
                       [-sin(psi)*cos(theta), cos(phi)*cos(psi) - sin(phi)*sin(theta)*sin(psi), sin(phi)*cos(psi) + sin(theta)*sin(psi)*cos(phi)],
                       [sin(theta), -sin(phi)*cos(theta), cos(phi)*cos(theta)]])
    rot = tf.transpose(rot, [2, 0, 1])
    trans = vector[:,:3]
    trans = np.reshape(trans, (np.shape(rot)[0], 3, 1))
    transform = tf.concat((rot, trans), axis = -1)
    extra = tf.tile(tf.constant([[[0., 0., 0., 1.]]], dtype = tf.double), (np.shape(rot)[0],1,1))
    transform = tf.concat((transform, extra), axis = -2)
    return transform

In [4]:
def t2v(mat):
    """converts transformation matrix to state vector"""
    if len( tf.shape(mat) ) == 2:
        mat = mat[None, :, :]
    R_sum = np.sqrt(( mat[:,0,0]**2 + mat[:,0,1]**2 + mat[:,1,2]**2 + mat[:,2,2]**2 ) / 2)
    phi = np.arctan2(-mat[:,1,2],mat[:,2,2])
    theta = np.arctan2(mat[:,0,2], R_sum)
    psi = np.arctan2(-mat[:,0,1], mat[:,0,0])
    angs = np.array([phi, theta, psi])
    vector = tf.concat((mat[:,:3,-1], angs.T), axis =1)
    return vector

In [ ]:
#NOTE: v2t() -> t2v() is returning to the same value only if I take inverse (for rotation) 
#                     and without inverse only (for translation). [T]^-1 @ [T] = I still
#Question: is this normal?  

# # test = np.array([1., 2., 3., 0.05, 0.00, -0.14])
# test = np.ones([2,6])
# test[1,:] = np.array([1., 2., 3., 0.003, 0.001, 0.3])
# T = v2t(test)
# # print("T: \n",T)
# vect_trans = t2v(T).numpy()[:,:3]
# vect_rot = t2v(tf.linalg.inv(T)).numpy()[:,3:]
# print(np.append(vect_trans, vect_rot, axis = 1))

# Least Squares Error Function


The optimial state vector, $x^*$, occurs where:

\begin{equation}
\Large
x^* = \arg\min_x \sum_{ij}^{} e^T_{ij}(x_i,x_j)\Omega_{ij}e_{ij}(x_i,x_j)
\end{equation}

$\Omega_{ij}$ is the information matrix associated with the odometry estimate that relates $i$ and $j$. $\Omega_{ij}$ is the inverse of the covariance matrix $\sigma_{ij}$

the error funcion for each connected node $i,j$ as a function of the state vector $x$ is defined as:

\begin{equation}
\Large
e_{ij}(x) = \text{t2v}(Z_{ij}^{-1} (X_j^{-1}X_i))
\end{equation}

\begin{equation}
\large
Z_{ij}^{-1} = \text{constraint (from measurement)}
\end{equation}

\begin{equation}
\large
(X_j^{-1}X_i) = x_i \text{ relative to }x_j \text{ given the current model of system} 
\end{equation}

### Important Note: $e_{ij}(x)$ only depends on $x_i$ and $x_j$ 

# Linearizing the System

\begin{equation}
\Large
e_{ij}(x + \Delta x) \approx e_{ij}(x) + J_{ij} \Delta x
\end{equation}

### Here, $J_{ij}$ is the the jacobian of $e_{ij}$ with respect to x

\begin{equation}
\Large
J_{ij} = \frac{\delta e_{ij}(x)}{\delta x} = \bigg{(} 0 \dots \frac{\delta e_{ij}(x_i)}{\delta x_i}
\dots \frac{\delta e_{ij}(x_j)}{\delta x_j} \dots 0 \bigg{)}
\end{equation}

<!-- \begin{equation}
\Large
J
\end{equation} -->

In [17]:
# TODO make function to generate J



### Similar to ICET, we can use Newton-Raphson to iteratively solve for small perterbations to the state vector $x$ that bring us towards a better solution

\begin{equation}
\Large
x \rightarrow x + \Delta x
\end{equation}

\begin{equation}
\Large
H \Delta x = -b
\end{equation}

\begin{equation}
\Large
b^T = \sum_{ij}^{}b_{ij} = \sum_{ij}^{}e_{ij}^T \Omega_{ij} J_{ij}
\end{equation}

\begin{equation}
\Large
H = \sum_{ij}^{}H_{ij} = \sum_{ij}^{}J_{ij}^T \Omega J_{ij}
\end{equation}

In [14]:
# Load test data
odometry_history = np.load("test_data/leddartech_pixset/T_vec_history.npy")
cov_estimate_history = np.load("test_data/leddartech_pixset/cov_vec_history.npy")
print(odometry_history[:20,-2:])
# print(cov_estimate_history)

[[ 2.  3.]
 [ 3.  4.]
 [ 2.  3.]
 [ 4. 11.]
 [ 3. 11.]
 [11. 14.]
 [11. 14.]
 [14. 17.]
 [14. 17.]
 [17. 20.]
 [17. 20.]
 [20. 23.]
 [20. 24.]
 [23. 26.]
 [24. 27.]
 [26. 29.]
 [27. 30.]
 [29. 32.]
 [30. 33.]
 [32. 35.]]


# TODO

Get derivitive of error function


# Questions

What is the Jacobian of the error function??

Is what we did in ICET to handle sparsity similar to a Choelesky Decomposition?


# Potential Contributions

Use ICET error covaraince estimation to demonstrate improvement in accuracy for Volpe Dataset 
